https://docs.llamaindex.ai/en/stable/examples/workflow/self_discover_workflow/

In [1]:
from llama_index.core.memory import ChatSummaryMemoryBuffer
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.llms.ollama import Ollama

In [2]:
import os
import mlflow
import mlflow.llama_index

In [3]:
log_dir = os.path.abspath("./mlruns")
mlflow.set_tracking_uri(f"file:{log_dir}")

In [4]:
log_dir

'/mnt/f/chatbot_ui/notebooks/mlruns'

In [5]:
mlflow.llama_index.autolog()

In [6]:
from llama_index.core.prompts import PromptTemplate

_REASONING_MODULES = [
    "1. How could I devise an experiment to help solve that problem?",
    "2. Make a list of ideas for solving this problem, and apply them one by one to the problem to see if any progress can be made.",
    "3. How could I measure progress on this problem?",
    "4. How can I simplify the problem so that it is easier to solve?",
    "5. What are the key assumptions underlying this problem?",
    "6. What are the potential risks and drawbacks of each solution?",
    "7. What are the alternative perspectives or viewpoints on this problem?",
    "8. What are the long-term implications of this problem and its solutions?",
    "9. How can I break down this problem into smaller, more manageable parts?",
    "10. Critical Thinking: This style involves analyzing the problem from different perspectives, questioning assumptions, and evaluating the evidence or information available. It focuses on logical reasoning, evidence-based decision-making, and identifying potential biases or flaws in thinking.",
    "11. Try creative thinking, generate innovative and out-of-the-box ideas to solve the problem. Explore unconventional solutions, thinking beyond traditional boundaries, and encouraging imagination and originality.",
    "12. Seek input and collaboration from others to solve the problem. Emphasize teamwork, open communication, and leveraging the diverse perspectives and expertise of a group to come up with effective solutions.",
    "13. Use systems thinking: Consider the problem as part of a larger system and understanding the interconnectedness of various elements. Focuses on identifying the underlying causes, feedback loops, and interdependencies that influence the problem, and developing holistic solutions that address the system as a whole.",
    "14. Use Risk Analysis: Evaluate potential risks, uncertainties, and tradeoffs associated with different solutions or approaches to a problem. Emphasize assessing the potential consequences and likelihood of success or failure, and making informed decisions based on a balanced analysis of risks and benefits.",
    "15. Use Reflective Thinking: Step back from the problem, take the time for introspection and self-reflection. Examine personal biases, assumptions, and mental models that may influence problem-solving, and being open to learning from past experiences to improve future approaches.",
    "16. What is the core issue or problem that needs to be addressed?",
    "17. What are the underlying causes or factors contributing to the problem?",
    "18. Are there any potential solutions or strategies that have been tried before? If yes, what were the outcomes and lessons learned?",
    "19. What are the potential obstacles or challenges that might arise in solving this problem?",
    "20. Are there any relevant data or information that can provide insights into the problem? If yes, what data sources are available, and how can they be analyzed?",
    "21. Are there any stakeholders or individuals who are directly affected by the problem? What are their perspectives and needs?",
    "22. What resources (financial, human, technological, etc.) are needed to tackle the problem effectively?",
    "23. How can progress or success in solving the problem be measured or evaluated?",
    "24. What indicators or metrics can be used?",
    "25. Is the problem a technical or practical one that requires a specific expertise or skill set? Or is it more of a conceptual or theoretical problem?",
    "26. Does the problem involve a physical constraint, such as limited resources, infrastructure, or space?",
    "27. Is the problem related to human behavior, such as a social, cultural, or psychological issue?",
    "28. Does the problem involve decision-making or planning, where choices need to be made under uncertainty or with competing objectives?",
    "29. Is the problem an analytical one that requires data analysis, modeling, or optimization techniques?",
    "30. Is the problem a design challenge that requires creative solutions and innovation?",
    "31. Does the problem require addressing systemic or structural issues rather than just individual instances?",
    "32. Is the problem time-sensitive or urgent, requiring immediate attention and action?",
    "33. What kinds of solution typically are produced for this kind of problem specification?",
    "34. Given the problem specification and the current best solution, have a guess about other possible solutions."
    "35. Let’s imagine the current best solution is totally wrong, what other ways are there to think about the problem specification?"
    "36. What is the best way to modify this current best solution, given what you know about these kinds of problem specification?"
    "37. Ignoring the current best solution, create an entirely new solution to the problem."
    "38. Let’s think step by step ."
    "39. Let’s make a step by step plan and implement it with good notation and explanation.",
]

_REASONING_MODULES = "\n".join(_REASONING_MODULES)

SELECT_PRMOPT_TEMPLATE = PromptTemplate(
    "Given the task: {task}, which of the following reasoning modules are relevant? Do not elaborate on why.\n\n {reasoning_modules}"
)

ADAPT_PROMPT_TEMPLATE = PromptTemplate(
    "Without working out the full solution, adapt the following reasoning modules to be specific to our task:\n{selected_modules}\n\nOur task:\n{task}"
)

IMPLEMENT_PROMPT_TEMPLATE = PromptTemplate(
    "Without working out the full solution, create an actionable reasoning structure for the task using these adapted reasoning modules:\n{adapted_modules}\n\nTask Description:\n{task}"
)

REASONING_PROMPT_TEMPLATE = PromptTemplate(
    "Using the following reasoning structure: {reasoning_structure}\n\nSolve this task, providing your final answer: {task}"
)

In [7]:
from llama_index.core.workflow import Event


class GetModulesEvent(Event):
    """Event to get modules."""

    task: str
    modules: str


class RefineModulesEvent(Event):
    """Event to refine modules."""

    task: str
    refined_modules: str


class ReasoningStructureEvent(Event):
    """Event to create reasoning structure."""

    task: str
    reasoning_structure: str

In [8]:
from llama_index.core.workflow import (
    Workflow,
    Context,
    StartEvent,
    StopEvent,
    step,
)
from llama_index.core.llms import LLM


class SelfDiscoverWorkflow(Workflow):
    """Self discover workflow."""

    @step
    async def get_modules(
        self, ctx: Context, ev: StartEvent
    ) -> GetModulesEvent:
        """Get modules step."""
        # get input data, store llm into ctx
        task = ev.get("task")
        llm: LLM = ev.get("llm")

        if task is None or llm is None:
            raise ValueError("'task' and 'llm' arguments are required.")

        await ctx.set("llm", llm)

        # format prompt and get result from LLM
        prompt = SELECT_PRMOPT_TEMPLATE.format(
            task=task, reasoning_modules=_REASONING_MODULES
        )
        result = llm.complete(prompt)

        return GetModulesEvent(task=task, modules=str(result))

    @step
    async def refine_modules(
        self, ctx: Context, ev: GetModulesEvent
    ) -> RefineModulesEvent:
        """Refine modules step."""
        task = ev.task
        modules = ev.modules
        llm: LLM = await ctx.get("llm")

        # format prompt and get result
        prompt = ADAPT_PROMPT_TEMPLATE.format(
            task=task, selected_modules=modules
        )
        result = llm.complete(prompt)

        return RefineModulesEvent(task=task, refined_modules=str(result))

    @step
    async def create_reasoning_structure(
        self, ctx: Context, ev: RefineModulesEvent
    ) -> ReasoningStructureEvent:
        """Create reasoning structures step."""
        task = ev.task
        refined_modules = ev.refined_modules
        llm: LLM = await ctx.get("llm")

        # format prompt, get result
        prompt = IMPLEMENT_PROMPT_TEMPLATE.format(
            task=task, adapted_modules=refined_modules
        )
        result = llm.complete(prompt)

        return ReasoningStructureEvent(
            task=task, reasoning_structure=str(result)
        )

    @step
    async def get_final_result(
        self, ctx: Context, ev: ReasoningStructureEvent
    ) -> StopEvent:
        """Gets final result from reasoning structure event."""
        task = ev.task
        reasoning_structure = ev.reasoning_structure
        llm: LLM = await ctx.get("llm")

        # format prompt, get res
        prompt = REASONING_PROMPT_TEMPLATE.format(
            task=task, reasoning_structure=reasoning_structure
        )
        result = llm.complete(prompt)

        return StopEvent(result=result)

In [9]:
#model = "qwen3:14b"
model = 'qwen3:14b'
tokenizer_model = "Qwen/Qwen3-14B"
context_window = 1000
#tokenizer = AutoTokenizer.from_pretrained(tokenizer_model)

llm = Ollama(
    model=model,
    request_timeout=120.0,
    thinking=False,
    context_window=context_window,
)

In [10]:
workflow = SelfDiscoverWorkflow()
llm = llm

In [11]:

from IPython.display import display, Markdown

task = """Michael has 15 oranges. He gives 4 oranges to his brother and trades 3 oranges for 6 apples with his neighbor. 
Later in the day, he realizes some of his oranges are spoiled, so he discards 2 of them. 
Then, Michael goes to the market and buys 12 more oranges and 5 more apples. 
If Michael decides to give 2 apples to his friend, how many oranges and apples does Michael have now?"""
result = await workflow.run(task=task, llm=llm)
display(Markdown(str(result)))

2025/06/22 15:58:23 WARNING mlflow.tracing.processor.mlflow_v2: Creating a trace within the default experiment with id '0'. It is strongly recommended to not use the default experiment to log traces due to ambiguous search results and probable performance issues over time due to directory table listing performance degradation with high volumes of directories within a specific path. To avoid performance and disambiguation issues, set the experiment for your environment using `mlflow.set_experiment()` API.


Let's go through the problem step by step using the reasoning structure provided:

---

### **Step 1: Initial Count (Module 1)**
- **Oranges:** 15  
- **Apples:** 0

---

### **Step 2: Giving Oranges to Brother (Module 2)**
- Michael gives **4 oranges** to his brother.
- **Oranges:** 15 - 4 = **11**  
- **Apples:** Still **0**

---

### **Step 3: Trading Oranges for Apples (Module 3)**
- Michael trades **3 oranges** for **6 apples**.
- **Oranges:** 11 - 3 = **8**  
- **Apples:** 0 + 6 = **6**

---

### **Step 4: Discarding Spoiled Oranges (Module 4)**
- Michael discards **2 spoiled oranges**.
- **Oranges:** 8 - 2 = **6**  
- **Apples:** Still **6**

---

### **Step 5: Buying More Oranges and Apples (Module 5)**
- Michael buys **12 oranges** and **5 apples**.
- **Oranges:** 6 + 12 = **18**  
- **Apples:** 6 + 5 = **11**

---

### **Step 6: Giving Apples to Friend (Module 6)**
- Michael gives **2 apples** to his friend.
- **Oranges:** Still **18**  
- **Apples:** 11 - 2 = **9**

---

### ✅ **Final Result:**
- **Oranges:** **18**  
- **Apples:** **9**

---

### ✅ **Answer:**
Michael now has **18 oranges** and **9 apples**.

In [12]:
from llama_index.utils.workflow import (
    draw_all_possible_flows,
    draw_most_recent_execution,
)

In [13]:
import os
cwd = os.getcwd()

In [14]:
filename = os.path.join(cwd,"self discover.html")
draw_all_possible_flows(SelfDiscoverWorkflow, filename=filename)

/mnt/f/chatbot_ui/notebooks/self discover.html


In [15]:
filename = os.path.join(cwd,"self discover recent.html")
draw_most_recent_execution(workflow, filename = filename)

/mnt/f/chatbot_ui/notebooks/self discover recent.html
